In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

from PIL import Image
import numpy as np
import pandas as pd 
import os 

In [ ]:
TEST_IMAGES_DIR = '/kaggle/input/plant-pathology-2021-fgvc8/test_images'
MODEL_PATH = '../input/plantmodels/efficientnet_b4.h5'
SUBMISSION_PATH = '/kaggle/working/submission.csv'

IMAGE_SIZE=(512,512)
LABELS = {0: 'complex', 1: 'frog_eye_leaf_spot', 2: 'healthy', 3: 'powdery_mildew', 4: 'rust', 5: 'scab'}

THRESHOLD = 0.43

In [ ]:
dependencies = {
    'Addons>F1Score': tfa.metrics.F1Score
}
model = tf.keras.models.load_model(MODEL_PATH, custom_objects=dependencies)

In [ ]:
submissions = []

for image_name in os.listdir(TEST_IMAGES_DIR):
    print(f"Running inference for file: {image_name}")
    image_path = os.path.join(TEST_IMAGES_DIR, image_name)
    image = Image.open(image_path)
    image = image.resize(IMAGE_SIZE)
    assert image.mode == 'RGB'
    
    np_image = np.expand_dims(np.array(image), axis=0)
    
    predctions = model.predict(np_image).ravel()
    arg_wheres = np.argwhere(predctions > THRESHOLD).ravel()
    
    if not arg_wheres.size:
        arg_wheres = np.argmax(predctions).ravel()
    
    classes = [LABELS[arg] for arg in arg_wheres]
    print(f'Detected classes: {classes}')
    prediction_string =  ' '.join(classes)
    submissions.append({'image': image_name, 'labels': prediction_string})
submission_df = pd.DataFrame(submissions)
submission_df.to_csv(SUBMISSION_PATH, index=False)